In [1]:
import lightning as L
import torch
import torch.nn as nn
import torch.nn.functional as F
from timm import create_model
from qat.export.utils import replace_module_by_name, fetch_module_by_name



/usr/local/lib/python3.10/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# from torch.jit import Final
# from timm.layers import use_fused_attn
# class Attention2(nn.Module):
#     fused_attn: Final[bool]
#     def __init__(
#             self,
#             dim,
#             num_heads=8,
#             qkv_bias=False,
#             qk_norm=False,
#             attn_drop=0.,
#             proj_drop=0.,
#             norm_layer=nn.LayerNorm,
#     ):
#         super().__init__()
#         assert dim % num_heads == 0, 'dim should be divisible by num_heads'
#         self.num_heads = num_heads
#         self.head_dim = dim // num_heads
#         self.scale = self.head_dim ** -0.5
#         self.fused_attn = use_fused_attn()

#         # self.qkv = nn.Linear(dim, dim * 3, bias=qkv_bias)
#         self.q_linear = nn.Linear(dim, dim, bias=qkv_bias)
#         self.k_linear = nn.Linear(dim, dim, bias=qkv_bias)
#         self.v_linear = nn.Linear(dim, dim, bias=qkv_bias)

#         self.q_norm = norm_layer(self.head_dim) if qk_norm else nn.Identity()
#         self.k_norm = norm_layer(self.head_dim) if qk_norm else nn.Identity()
#         self.attn_drop = nn.Dropout(attn_drop)
#         self.proj = nn.Linear(dim, dim)
#         self.proj_drop = nn.Dropout(proj_drop)

#     def forward(self, x):
#         B, N, C = x.shape
#         # qkv = self.qkv(x).reshape(B, N, 3, self.num_heads, self.head_dim).permute(2, 0, 3, 1, 4) # (3, B, heads, N, d)
#         # q, k, v = qkv.unbind(0)
#         q = self.q_linear(x).reshape(B, N, self.num_heads, self.head_dim).permute(0, 2, 1, 3)
#         k = self.k_linear(x).reshape(B, N, self.num_heads, self.head_dim).permute(0, 2, 1, 3)
#         v = self.v_linear(x).reshape(B, N, self.num_heads, self.head_dim).permute(0, 2, 1, 3)

#         q, k = self.q_norm(q), self.k_norm(k)

#         if self.fused_attn:
#             x = F.scaled_dot_product_attention(
#                 q, k, v,
#                 dropout_p=self.attn_drop.p,
#             )
#         else:
#             q = q * self.scale
#             attn = q @ k.transpose(-2, -1)
#             attn = attn.softmax(dim=-1)
#             attn = self.attn_drop(attn)
#             x = attn @ v

#         x = x.transpose(1, 2).reshape(B, N, C)
#         x = self.proj(x)
#         x = self.proj_drop(x)
#         return x

In [2]:
from torch.jit import Final
from timm.layers import use_fused_attn
class Attention3(nn.Module):
    fused_attn: Final[bool]
    def __init__(
            self,
            dim,
            num_heads=8,
            qkv_bias=False,
            qk_norm=False,
            attn_drop=0.,
            proj_drop=0.,
            norm_layer=nn.LayerNorm,
    ):
        super().__init__()
        assert dim % num_heads == 0, 'dim should be divisible by num_heads'
        self.num_heads = num_heads
        self.head_dim = dim // num_heads
        self.scale = self.head_dim ** -0.5
        self.fused_attn = use_fused_attn()

        self.qkv = nn.Linear(dim, dim * 3, bias=qkv_bias)
        self.q_norm = norm_layer(self.head_dim) if qk_norm else nn.Identity()
        self.k_norm = norm_layer(self.head_dim) if qk_norm else nn.Identity()
        self.attn_drop = nn.Dropout(attn_drop)
        self.proj = nn.Linear(dim, dim)
        self.proj_drop = nn.Dropout(proj_drop)
        self.register_parameter(
            "inverse_temperature_logit",
            nn.Parameter(torch.randn(1))
        )
    def forward(self, x):
        B, N, C = x.shape
        qkv = self.qkv(x).reshape(B, N, 3, self.num_heads, self.head_dim).permute(2, 0, 3, 1, 4)
        q, k, v = qkv.unbind(0)
        q, k = self.q_norm(q), self.k_norm(k)

        if self.fused_attn:
            x = F.scaled_dot_product_attention(
                q, k, v,
                dropout_p=self.attn_drop.p,
            )
        else:
            q = q * self.scale
            attn = q @ k.transpose(-2, -1)
            attn = attn.softmax(dim=-1)
            attn_argmax = attn.argmax(dim=-1)
            attn = self.attn_drop(attn)
            output = attn - (attn - attn_argmax).detach() # output = real_output - (real_output - quantized_output).detach()
            x = output @ v

        x = x.transpose(1, 2).reshape(B, N, C)
        x = self.proj(x)
        x = self.proj_drop(x)
        return x

In [5]:
# model_name = "deit3_huge_patch14_224.fb_in22k_ft_in1k"
# model_name = "deit3_base_patch16_224.fb_in22k_ft_in1k"
model_name = 'deit3_small_patch16_224.fb_in22k_ft_in1k'
model = create_model(model_name, pretrained=True)

In [7]:
summary(model1, (1, 3, 224, 224))

Layer (type:depth-idx)                   Output Shape              Param #
VisionTransformer                        [1, 1000]                 75,648
├─PatchEmbed: 1-1                        [1, 196, 384]             --
│    └─Conv2d: 2-1                       [1, 384, 14, 14]          295,296
│    └─Identity: 2-2                     [1, 196, 384]             --
├─Dropout: 1-2                           [1, 197, 384]             --
├─Identity: 1-3                          [1, 197, 384]             --
├─Identity: 1-4                          [1, 197, 384]             --
├─Sequential: 1-5                        [1, 197, 384]             --
│    └─Block: 2-3                        [1, 197, 384]             --
│    │    └─LayerNorm: 3-1               [1, 197, 384]             768
│    │    └─Attention: 3-2               [1, 197, 384]             591,360
│    │    └─LayerScale: 3-3              [1, 197, 384]             384
│    │    └─Identity: 3-4                [1, 197, 384]             --

In [6]:
model1

VisionTransformer(
  (patch_embed): PatchEmbed(
    (proj): Conv2d(3, 384, kernel_size=(16, 16), stride=(16, 16))
    (norm): Identity()
  )
  (pos_drop): Dropout(p=0.0, inplace=False)
  (patch_drop): Identity()
  (norm_pre): Identity()
  (blocks): Sequential(
    (0): Block(
      (norm1): LayerNorm((384,), eps=1e-06, elementwise_affine=True)
      (attn): Attention(
        (qkv): Linear(in_features=384, out_features=1152, bias=True)
        (q_norm): Identity()
        (k_norm): Identity()
        (attn_drop): Dropout(p=0.0, inplace=False)
        (proj): Linear(in_features=384, out_features=384, bias=True)
        (proj_drop): Dropout(p=0.0, inplace=False)
      )
      (ls1): LayerScale()
      (drop_path1): Identity()
      (norm2): LayerNorm((384,), eps=1e-06, elementwise_affine=True)
      (mlp): Mlp(
        (fc1): Linear(in_features=384, out_features=1536, bias=True)
        (act): GELU(approximate='none')
        (drop1): Dropout(p=0.0, inplace=False)
        (norm): Identit

In [5]:
# settings = {
#     "attn": 384,
# }

# for i in range(0, 12): 
#     for name in settings:
#         layer = model.blocks[i]
#         module = fetch_module_by_name(layer, name)
#         attn = Attention3(
#                 dim=settings[name], # 為什麼從attn module拿不到 head_dim?
#                 num_heads=getattr(module, "num_heads"),
#                 qkv_bias=True,
#                 qk_norm=False
#             )
        
#         qkv_weight = module.qkv.weight.data
#         qkv_bias = module.qkv.bias.data
#         dim = qkv_weight.shape[0] // 3
#         q_weight, k_weight, v_weight = qkv_weight[:dim], qkv_weight[dim:2*dim], qkv_weight[2*dim:]
#         # print(q_weight.shape)
#         q_bias, k_bias, v_bias = qkv_bias[:dim], qkv_bias[dim:2*dim], qkv_bias[2*dim:]
#         attn.q_linear.weight.data = q_weight
#         attn.q_linear.bias.data = q_bias
#         attn.k_linear.weight.data = k_weight
#         attn.k_linear.bias.data = k_bias
#         attn.v_linear.weight.data = v_weight
#         attn.v_linear.bias.data = v_bias
#         attn.attn_drop = module.attn_drop
#         attn.proj = module.proj
#         attn.proj_drop = module.proj_drop
#         replace_module_by_name(layer, name, attn)

In [4]:
settings = {
    "attn": 384, # dimension of the model
}

for i in range(0, 12): 
    for name in settings:
        layer = model.blocks[i]
        module = fetch_module_by_name(layer, name)
        attn = Attention3(
                dim=settings[name],
                num_heads=getattr(module, "num_heads"),
                qkv_bias=True,
                qk_norm=False
            )
        attn.inverse_temperature_logit.data.copy_(
                torch.tensor(10)
            )
        qkv_weight = module.qkv.weight.data
        qkv_bias = module.qkv.bias.data
        attn.qkv.weight.data = qkv_weight
        attn.qkv.bias.data = qkv_bias
        attn.attn_drop = module.attn_drop
        attn.proj = module.proj
        attn.proj_drop = module.proj_drop
        replace_module_by_name(layer, name, attn)

In [6]:
model.blocks[0].attn.inverse_temperature_logit

Parameter containing:
tensor([10.], requires_grad=True)

In [ ]:
torch.save(model, f'{model_name}.pth')

In [1]:
import timm
timm.list_models("resmlp*")

/usr/local/lib/python3.10/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


['resmlp_12_224', 'resmlp_24_224', 'resmlp_36_224', 'resmlp_big_24_224']

In [ ]:
import timm
timm.list_models("resmlp*")

In [9]:
model.blocks[0]

ResBlock(
  (norm1): Affine()
  (linear_tokens): Linear(in_features=196, out_features=196, bias=True)
  (drop_path): Identity()
  (norm2): Affine()
  (mlp_channels): Mlp(
    (fc1): Linear(in_features=384, out_features=1536, bias=True)
    (act): GELU(approximate='none')
    (drop1): Dropout(p=0.0, inplace=False)
    (norm): Identity()
    (fc2): Linear(in_features=1536, out_features=384, bias=True)
    (drop2): Dropout(p=0.0, inplace=False)
  )
)

In [10]:
def gcd(a, b):
    while b:
        a, b = b, a % b
    return abs(a)

# 測試
x = 196
y = 384
print(f"The GCD of {x} and {y} is {gcd(x, y)}")


The GCD of 196 and 384 is 4


In [ ]:
from torchinfo import summary
from timm import create_model
model_name = 'resmlp_12_224.fb_in1k'
model = create_model(model_name, pretrained=True)
summary(model,(1,3,224,224)) 

In [4]:
model

MlpMixer(
  (stem): PatchEmbed(
    (proj): Conv2d(3, 384, kernel_size=(16, 16), stride=(16, 16))
    (norm): Identity()
  )
  (blocks): Sequential(
    (0): ResBlock(
      (norm1): Affine()
      (linear_tokens): Linear(in_features=196, out_features=196, bias=True)
      (drop_path): Identity()
      (norm2): Affine()
      (mlp_channels): Mlp(
        (fc1): Linear(in_features=384, out_features=1536, bias=True)
        (act): GELU(approximate='none')
        (drop1): Dropout(p=0.0, inplace=False)
        (norm): Identity()
        (fc2): Linear(in_features=1536, out_features=384, bias=True)
        (drop2): Dropout(p=0.0, inplace=False)
      )
    )
    (1): ResBlock(
      (norm1): Affine()
      (linear_tokens): Linear(in_features=196, out_features=196, bias=True)
      (drop_path): Identity()
      (norm2): Affine()
      (mlp_channels): Mlp(
        (fc1): Linear(in_features=384, out_features=1536, bias=True)
        (act): GELU(approximate='none')
        (drop1): Dropout(p=0.

In [ ]:
from torchinfo import summary
model_name = 'deit3_huge_patch14_224.fb_in22k_ft_in1k'
model = create_model(model_name, pretrained=True)
summary(model,(1,3,224,224)) 